In [2]:
import pandas as pd

# Dictionnaire associant tickers et noms complets des commodities
commodities = {
    "GC=F": "Gold Futures",
    "SI=F": "Silver Futures",
    "CL=F": "Crude Oil Futures",
    "NG=F": "Natural Gas Futures",
    "HG=F": "Copper Futures",
    "ZC=F": "Corn Futures",
    "ZW=F": "Wheat Futures",
    "ZL=F": "Soybean Futures",
    "KC=F": "Coffee Futures",
    "CT=F": "Cotton Futures",
    "SB=F": "Sugar Futures",
    "LE=F": "Live Cattle Futures",
    "HE=F": "Heating Oil Futures",
    "RB=F": "RBOB Gasoline Futures",
    "PL=F": "Platinum Futures",
    "PA=F": "Palladium Futures",
    "CC=F": "Cocoa Futures"
}

# Création du DataFrame à partir du dictionnaire
df_commodities = pd.DataFrame(list(commodities.items()), columns=["Ticker", "Name"])

# Ajout de la colonne "Type" avec la valeur "FUTURE" pour chaque ligne
df_commodities["Type"] = "FUTURE"


In [3]:
import pandas as pd

# Dictionnaire associant tickers et noms complets pour le Forex
forex = {
    "EURUSD=X": "Euro / US Dollar",
    "GBPUSD=X": "British Pound / US Dollar",
    "USDJPY=X": "US Dollar / Japanese Yen",
    "AUDUSD=X": "Australian Dollar / US Dollar",
    "USDCAD=X": "US Dollar / Canadian Dollar",
    "USDCHF=X": "US Dollar / Swiss Franc",
    "NZDUSD=X": "New Zealand Dollar / US Dollar",
    "EURGBP=X": "Euro / British Pound",
    "EURJPY=X": "Euro / Japanese Yen",
    "GBPJPY=X": "British Pound / Japanese Yen",
    "AUDJPY=X": "Australian Dollar / Japanese Yen",
    "EURAUD=X": "Euro / Australian Dollar",
    "GBPCHF=X": "British Pound / Swiss Franc",
    "CHFJPY=X": "Swiss Franc / Japanese Yen",
    "CADJPY=X": "Canadian Dollar / Japanese Yen",
    "NZDJPY=X": "New Zealand Dollar / Japanese Yen",
    "EURCAD=X": "Euro / Canadian Dollar",
    "GBPCAD=X": "British Pound / Canadian Dollar",
    "AUDCAD=X": "Australian Dollar / Canadian Dollar",
    "AUDCHF=X": "Australian Dollar / Swiss Franc",
    "NZDCHF=X": "New Zealand Dollar / Swiss Franc"
}

# Création du DataFrame à partir du dictionnaire
df_forex = pd.DataFrame(list(forex.items()), columns=["Ticker", "Name"])

# Ajout de la colonne "Type" avec la valeur "FOREX" pour chaque ligne
df_forex["Type"] = "FOREX"

In [7]:
df = pd.read_csv('symbols_valid_meta.csv')[['Symbol','Security Name','ETF']]

In [18]:
# Création du DataFrame pour les actions (ETF == "N")
df_stock = df[df["ETF"] == "N"].copy()
df_stock = df_stock.rename(columns={'Symbol':'Ticker','Security Name':'Name','ETF':'Type'})
df_stock['Type'] = 'STOCK'

# Création du DataFrame pour les ETF (ETF == "Y")
df_etf = df[df["ETF"] == "Y"].copy()
df_etf = df_etf.rename(columns={'Symbol':'Ticker','Security Name':'Name','ETF':'Type'})
df_etf['Type'] = 'ETF'

In [21]:
df = pd.concat([df_forex,df_commodities,df_etf,df_stock])

In [23]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Initialisation du modèle (ici "all-MiniLM-L6-v2", vous pouvez choisir un autre modèle)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Calcul des embeddings pour chaque valeur de la colonne "Name"
df['embeddings'] = df['Name'].apply(lambda name: model.encode(name))


/Users/loubrunet/miniconda3/envs/var_project/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
df.to_csv('tickers_data.csv')

In [26]:
df

,Ticker,Name,Type,embeddings
0,EURUSD=X,Euro / US Dollar,FOREX,"[0.028471347, 0.008860748, -0.07475, 0.0190373..."
1,GBPUSD=X,British Pound / US Dollar,FOREX,"[0.027380986, -0.049044646, -0.06961119, -0.00..."
2,USDJPY=X,US Dollar / Japanese Yen,FOREX,"[-0.0015552728, 0.013238107, -0.06042375, 0.04..."
3,AUDUSD=X,Australian Dollar / US Dollar,FOREX,"[0.008561188, 0.014089047, -0.095514566, 0.032..."
4,USDCAD=X,US Dollar / Canadian Dollar,FOREX,"[0.011619025, 0.0068328083, -0.04130929, 0.013..."
...,...,...,...,...
8044,ZUO,"Zuora, Inc. Class A Common Stock",STOCK,"[-0.050629795, 0.027944237, -0.018637272, 0.04..."
8045,ZVO,Zovio Inc. - Common Stock,STOCK,"[-0.08400878, 0.034375172, -0.095743015, 0.024..."
8046,ZYME,Zymeworks Inc. Common Shares,STOCK,"[-0.07689729, -0.0052898377, 0.0014849417, 0.0..."
8047,ZYNE,"Zynerba Pharmaceuticals, Inc. - Common Stock",STOCK,"[-0.04748842, -0.0030635518, -0.0722399, -0.00..."


In [27]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def get_top10_assets(query_text: str, data: pd.DataFrame, model: SentenceTransformer) -> pd.DataFrame:
    """
    Retourne les 10 actifs dont le nom est le plus proche du texte de requête.

    Parameters:
        query_text (str): Le texte de la requête.
        data (pd.DataFrame): DataFrame contenant les actifs et leurs embeddings (dans la colonne 'embeddings').
        model (SentenceTransformer): Modèle SentenceTransformer pour encoder la requête.

    Returns:
        pd.DataFrame: Les 10 actifs les plus proches.
    """
    # Calcul de l'embedding pour le texte de la requête
    query_embedding = model.encode(query_text)

    # Calcul de la similarité cosinus entre l'embedding de la requête et chaque embedding de l'actif
    # On transforme la colonne 'embeddings' en tableau numpy pour un calcul vectorisé
    asset_embeddings = np.vstack(data['embeddings'].values)
    similarities = cosine_similarity(asset_embeddings, [query_embedding]).squeeze()

    # Ajout des scores de similarité au DataFrame
    data = data.copy()  # pour ne pas modifier l'original
    data['similarity'] = similarities

    # Tri par similarité décroissante et sélection des 10 meilleurs
    top10 = data.sort_values(by='similarity', ascending=False).head(10)
    return top10

# Exemple d'utilisation de la fonction
query = "Euro Dollar"
top_assets = get_top10_assets(query, df, model)
print(top_assets[['Ticker', 'Name', 'Type', 'similarity']])

        Ticker                       Name   Type  similarity
0     EURUSD=X           Euro / US Dollar  FOREX    0.914299
16    EURCAD=X     Euro / Canadian Dollar  FOREX    0.792725
11    EURAUD=X   Euro / Australian Dollar  FOREX    0.785393
7     EURGBP=X       Euro / British Pound  FOREX    0.740863
8     EURJPY=X        Euro / Japanese Yen  FOREX    0.686752
5     USDCHF=X    US Dollar / Swiss Franc  FOREX    0.634226
2325      EUFX       ProShares Short Euro  STOCK    0.592550
2329       EUO  ProShares UltraShort Euro    ETF    0.587500
1     GBPUSD=X  British Pound / US Dollar  FOREX    0.585678
7353       ULE       ProShares Ultra Euro    ETF    0.584555


In [28]:
top_assets

,Ticker,Name,Type,embeddings,similarity
0,EURUSD=X,Euro / US Dollar,FOREX,"[0.028471347, 0.008860748, -0.07475, 0.0190373...",0.914299
16,EURCAD=X,Euro / Canadian Dollar,FOREX,"[0.022548126, 0.0037843348, -0.019853601, 0.00...",0.792725
11,EURAUD=X,Euro / Australian Dollar,FOREX,"[0.018708246, 0.021835392, -0.0825525, 0.03207...",0.785393
7,EURGBP=X,Euro / British Pound,FOREX,"[0.03734842, -0.013618621, -0.07262709, -0.007...",0.740863
8,EURJPY=X,Euro / Japanese Yen,FOREX,"[-0.0032634672, 0.036472913, -0.05968292, 0.04...",0.686752
5,USDCHF=X,US Dollar / Swiss Franc,FOREX,"[-0.018100714, -0.0028878953, -0.062068358, 0....",0.634226
2325,EUFX,ProShares Short Euro,STOCK,"[0.045869198, 0.06642851, -0.047211386, -0.046...",0.592550
2329,EUO,ProShares UltraShort Euro,ETF,"[0.030384408, -0.031079307, -0.103039056, -0.0...",0.587500
1,GBPUSD=X,British Pound / US Dollar,FOREX,"[0.027380986, -0.049044646, -0.06961119, -0.00...",0.585678
7353,ULE,ProShares Ultra Euro,ETF,"[0.016023379, -0.006424614, -0.097034514, -0.0...",0.584555


In [32]:
second_df = pd.read_csv('World-Stock-Prices-Dataset.csv')[['Brand_Name','Ticker']]

In [35]:
second_df['Brand_Name'] = second_df['Brand_Name'].apply(lambda name: name.capitalize())

In [38]:
second_df = second_df.rename(columns={'Brand_Name':'Name'})

In [40]:
second_df['Type'] = 'STOCK'

In [42]:
df

,Ticker,Name,Type,embeddings
0,EURUSD=X,Euro / US Dollar,FOREX,"[0.028471347, 0.008860748, -0.07475, 0.0190373..."
1,GBPUSD=X,British Pound / US Dollar,FOREX,"[0.027380986, -0.049044646, -0.06961119, -0.00..."
2,USDJPY=X,US Dollar / Japanese Yen,FOREX,"[-0.0015552728, 0.013238107, -0.06042375, 0.04..."
3,AUDUSD=X,Australian Dollar / US Dollar,FOREX,"[0.008561188, 0.014089047, -0.095514566, 0.032..."
4,USDCAD=X,US Dollar / Canadian Dollar,FOREX,"[0.011619025, 0.0068328083, -0.04130929, 0.013..."
...,...,...,...,...
8044,ZUO,"Zuora, Inc. Class A Common Stock",STOCK,"[-0.050629795, 0.027944237, -0.018637272, 0.04..."
8045,ZVO,Zovio Inc. - Common Stock,STOCK,"[-0.08400878, 0.034375172, -0.095743015, 0.024..."
8046,ZYME,Zymeworks Inc. Common Shares,STOCK,"[-0.07689729, -0.0052898377, 0.0014849417, 0.0..."
8047,ZYNE,"Zynerba Pharmaceuticals, Inc. - Common Stock",STOCK,"[-0.04748842, -0.0030635518, -0.0722399, -0.00..."


In [43]:
second_df['embeddings'] = second_df['Name'].apply(lambda name: model.encode(name))

In [44]:
second_df

,Name,Ticker,Type,embeddings
0,Peloton,PTON,STOCK,"[-0.08778438, 0.07176341, -0.048017673, -0.092..."
1,Crocs,CROX,STOCK,"[-0.031235097, -0.0007110486, -0.076940015, 0...."
2,The coca-cola company,KO,STOCK,"[-0.09234252, -0.0169516, -0.012214464, 0.0393..."
3,Netflix,NFLX,STOCK,"[-0.04787394, -0.1562092, -0.048543714, -0.015..."
4,Foot locker,FL,STOCK,"[-0.12066561, 0.08526429, -0.03607174, -0.0095..."
...,...,...,...,...
303870,Microsoft,MSFT,STOCK,"[-0.04431741, -0.010996484, -0.042527903, -0.0..."
303871,Amd,AMD,STOCK,"[-0.03047849, 0.059754357, 0.05613379, 0.04350..."
303872,Nike,NKE,STOCK,"[-0.08180545, 0.0154199675, 0.003472329, 0.005..."
303873,Hershey company,HSY,STOCK,"[-0.05239339, -0.039327487, 0.028091233, 0.039..."


In [45]:
def get_top10_assets(query_text: str, data: pd.DataFrame, model: SentenceTransformer) -> pd.DataFrame:
    """
    Retourne les 10 actifs dont le nom est le plus proche du texte de requête.
    """
    query_embedding = model.encode(query_text)
    asset_embeddings = np.vstack(data['embeddings'].values)
    similarities = cosine_similarity(asset_embeddings, [query_embedding]).squeeze()
    data = data.copy()
    data['similarity'] = similarities
    top10 = data.sort_values(by='similarity', ascending=False).head(10)
    return top10

In [53]:
second_df.Name.nunique()

61

In [47]:
get_top10_assets('apple',second_df,SentenceTransformer('all-MiniLM-L6-v2'))

,Name,Ticker,Type,embeddings,similarity
70934,Apple,AAPL,STOCK,"[-0.006138524, 0.031011775, 0.06479362, 0.0109...",1.0
29288,Apple,AAPL,STOCK,"[-0.006138524, 0.031011775, 0.06479362, 0.0109...",1.0
17336,Apple,AAPL,STOCK,"[-0.006138524, 0.031011775, 0.06479362, 0.0109...",1.0
179354,Apple,AAPL,STOCK,"[-0.006138524, 0.031011775, 0.06479362, 0.0109...",1.0
286106,Apple,AAPL,STOCK,"[-0.006138524, 0.031011775, 0.06479362, 0.0109...",1.0
29263,Apple,AAPL,STOCK,"[-0.006138524, 0.031011775, 0.06479362, 0.0109...",1.0
252090,Apple,AAPL,STOCK,"[-0.006138524, 0.031011775, 0.06479362, 0.0109...",1.0
128420,Apple,AAPL,STOCK,"[-0.006138524, 0.031011775, 0.06479362, 0.0109...",1.0
200375,Apple,AAPL,STOCK,"[-0.006138524, 0.031011775, 0.06479362, 0.0109...",1.0
286081,Apple,AAPL,STOCK,"[-0.006138524, 0.031011775, 0.06479362, 0.0109...",1.0


In [54]:
import financedatabase as fd

# Initialize the Equities database
equities = fd.Equities()

# Obtain all data available excluding international exchanges
equities.select()

,name,summary,currency,sector,industry_group,industry,exchange,market,country,state,city,zipcode,website,market_cap,isin,cusip,figi,composite_figi,shareclass_figi
symbol,,,,,,,,,,,,,,,,,,,
A,"Agilent Technologies, Inc.","Agilent Technologies, Inc. provides applicatio...",USD,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Biotechnology,NYQ,New York Stock Exchange,United States,CA,Santa Clara,95051,http://www.agilent.com,Large Cap,US00846U1016,00846U101,BBG000C2V541,BBG000C2V3D6,BBG001SCTQY4
AA,Alcoa Corporation,"Alcoa Corporation, together with its subsidiar...",USD,Materials,Materials,Metals & Mining,NYQ,New York Stock Exchange,United States,PA,Pittsburgh,15212-5858,http://www.alcoa.com,Mid Cap,US0138721065,13872106,BBG00B3T3HK5,BBG00B3T3HD3,BBG00B3T3HF1
AAALF,Aareal Bank AG,"Aareal Bank AG, together with its subsidiaries...",USD,Financials,Banks,Banks,PNK,OTC Bulletin Board,Germany,NaN,Wiesbaden,65189,http://www.aareal-bank.com,Small Cap,US00254K1088,00254K108,NaN,NaN,NaN
AAALY,Aareal Bank AG,"Aareal Bank AG, together with its subsidiaries...",USD,Financials,Banks,Banks,PNK,OTC Bulletin Board,Germany,NaN,Wiesbaden,65189,http://www.aareal-bank.com,Small Cap,US00254K1088,00254K108,NaN,NaN,NaN
AABB,"Asia Broadband, Inc.","Asia Broadband Inc., through its subsidiary, A...",USD,Materials,Materials,Metals & Mining,PNK,OTC Bulletin Board,United States,NV,Las Vegas,89135,http://www.asiabroadbandinc.com,Micro Cap,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,Nano Labs Ltd Class A Ordinary Shares,NaN,USD,Information Technology,Semiconductors & Semiconductor Equipment,Semiconductors & Semiconductor Equipment,NMS,NASDAQ Global Select,China,NaN,NaN,NaN,NaN,Nano Cap,NaN,NaN,NaN,NaN,NaN
NaN,Nano Labs Ltd Class A Ordinary Shares,NaN,USD,Information Technology,Semiconductors & Semiconductor Equipment,Semiconductors & Semiconductor Equipment,NMS,NASDAQ Global Select,China,NaN,NaN,NaN,NaN,Nano Cap,NaN,NaN,NaN,NaN,NaN
NaN,Nano Labs Ltd Class A Ordinary Shares,NaN,USD,Information Technology,Semiconductors & Semiconductor Equipment,Semiconductors & Semiconductor Equipment,NMS,NASDAQ Global Select,China,NaN,NaN,NaN,NaN,Nano Cap,NaN,NaN,NaN,NaN,NaN


In [70]:
second_df = equities.select().reset_index()[["symbol","name"]].dropna().rename(columns={'symbol':'Ticker','name':'Name'})

In [72]:
second_df['Type'] = 'STOCK'

In [74]:
second_df['embeddings'] = second_df['Name'].apply(lambda name: model.encode(name))

In [78]:
final_df = pd.concat([df,second_df]).drop_duplicates(subset='Ticker')

In [89]:
final_df.to_csv('tickers_data.csv')

In [88]:
get_top10_assets('sap',final_df,SentenceTransformer('all-MiniLM-L6-v2'))

,Ticker,Name,Type,embeddings,similarity
18691,SAPGF,SAP SE,STOCK,"[-0.0362483, -0.07981501, -0.008086964, 0.0204...",0.904784
6296,SAP,SAP SE ADS,STOCK,"[-0.06862049, -0.068881, -0.030161535, 0.02663...",0.769142
20019,SPPJY,Sappi Limited,STOCK,"[-0.01122841, 0.00014704576, 0.005928153, 0.00...",0.709882
18692,SAPIF,Saputo Inc.,STOCK,"[-0.03897073, 0.014120918, -0.07481489, 0.0116...",0.600192
11802,JIH-UN,"Juniper Industrial Holdings, In",STOCK,"[-0.07259049, -0.083747566, -0.030294012, -0.0...",0.498266
11803,JIH-WT,"Juniper Industrial Holdings, In",STOCK,"[-0.07259049, -0.083747566, -0.030294012, -0.0...",0.498266
19643,SMNEY,Siemens Energy AG,STOCK,"[-0.14698191, 0.013308741, 0.0058764024, 0.031...",0.495952
19612,SMEGF,Siemens Energy AG,STOCK,"[-0.14698191, 0.013308741, 0.0058764024, 0.031...",0.495952
4837,CMRZF,Commerce Resources Corp.,STOCK,"[-0.055734735, 0.0064990236, -0.07660297, 0.03...",0.495102
20069,SQFL,SQL Technologies Corp.,STOCK,"[-0.045804717, -0.014207067, -0.093505695, 0.0...",0.493120


In [90]:
final_df

,Ticker,Name,Type,embeddings
0,EURUSD=X,Euro / US Dollar,FOREX,"[0.028471347, 0.008860748, -0.07475, 0.0190373..."
1,GBPUSD=X,British Pound / US Dollar,FOREX,"[0.027380986, -0.049044646, -0.06961119, -0.00..."
2,USDJPY=X,US Dollar / Japanese Yen,FOREX,"[-0.0015552728, 0.013238107, -0.06042375, 0.04..."
3,AUDUSD=X,Australian Dollar / US Dollar,FOREX,"[0.008561188, 0.014089047, -0.095514566, 0.032..."
4,USDCAD=X,US Dollar / Canadian Dollar,FOREX,"[0.011619025, 0.0068328083, -0.04130929, 0.013..."
...,...,...,...,...
23900,ZYRX,"Zyrox Mining International, Inc.",STOCK,"[-0.13603753, 0.12183122, -0.034411352, 0.0149..."
23902,ZZHGF,"ZhongAn Online P & C Insurance Co., Ltd.",STOCK,"[-0.091545105, 0.12819937, -0.06316054, -0.005..."
23903,ZZHGY,"ZhongAn Online P & C Insurance Co., Ltd.",STOCK,"[-0.091545105, 0.12819937, -0.06316054, -0.005..."
23904,ZZLL,"ZZLL Information Technology, Inc.",STOCK,"[-0.13600339, 0.12652822, -0.056081597, 0.0099..."


In [118]:
import json
df = final_df.copy()

In [119]:
df['embeddings'] = df['embeddings'].apply(lambda arr: json.dumps(arr.tolist()))

In [120]:
df.to_csv('tickers_data.csv',index=False)

In [117]:
test_df

,Unnamed: 0,Ticker,Name,Type,embeddings
0,0,EURUSD=X,Euro / US Dollar,FOREX,"[0.02847134694457054, 0.008860748261213303, -0..."
1,1,GBPUSD=X,British Pound / US Dollar,FOREX,"[0.027380986139178276, -0.0490446463227272, -0..."
2,2,USDJPY=X,US Dollar / Japanese Yen,FOREX,"[-0.0015552728436887264, 0.013238106854259968,..."
3,3,AUDUSD=X,Australian Dollar / US Dollar,FOREX,"[0.008561188355088234, 0.014089046977460384, -..."
4,4,USDCAD=X,US Dollar / Canadian Dollar,FOREX,"[0.01161902491003275, 0.006832808256149292, -0..."
...,...,...,...,...,...
26549,23900,ZYRX,"Zyrox Mining International, Inc.",STOCK,"[-0.13603752851486206, 0.12183122336864471, -0..."
26550,23902,ZZHGF,"ZhongAn Online P & C Insurance Co., Ltd.",STOCK,"[-0.09154510498046875, 0.12819936871528625, -0..."
26551,23903,ZZHGY,"ZhongAn Online P & C Insurance Co., Ltd.",STOCK,"[-0.09154510498046875, 0.12819936871528625, -0..."
26552,23904,ZZLL,"ZZLL Information Technology, Inc.",STOCK,"[-0.13600338995456696, 0.12652821838855743, -0..."


In [111]:
test_df = pd.read_csv('test.csv')
test_df['embeddings'] = test_df['embeddings'].apply(lambda s: np.array(json.loads(s)))

In [113]:
test_df['embeddings'] = test_df['embeddings'].apply(lambda s: np.array(json.loads(s)))

In [116]:
test_df.embeddings[0]

array([ 2.84713469e-02,  8.86074826e-03, -7.47499987e-02,  1.90373622e-02,
        2.54122429e-02, -7.14884922e-02, -6.29169075e-03,  3.33614647e-02,
        6.05819263e-02, -5.78751266e-02, -1.16217220e-02, -6.16319589e-02,
       -3.14545035e-02,  5.86376563e-02, -1.74181070e-02, -7.02636018e-02,
       -6.53280923e-03,  1.97970513e-02, -5.56756072e-02,  3.42304930e-02,
        1.44586358e-02, -5.04535027e-02, -4.93305102e-02,  1.87152438e-02,
        2.98604257e-02,  4.01163027e-02,  6.79352134e-02, -3.80760506e-02,
        5.97673515e-03, -1.90130249e-02, -2.91799027e-02, -4.72847223e-02,
       -3.61585505e-02, -7.92897418e-02, -9.89605114e-03, -1.34576440e-01,
        7.70159662e-02,  3.18283215e-03, -1.79356448e-02, -5.84582053e-03,
       -4.94555011e-02, -9.02727246e-02,  1.35159222e-02, -3.83593142e-02,
       -2.15562377e-02,  6.88964501e-02, -1.74085796e-02,  1.53199613e-01,
       -2.13943422e-02,  7.14501441e-02,  1.09211035e-01,  1.94426347e-02,
       -5.47160730e-02,  

In [97]:
test_df = pd.read_csv('test.csv')
df['embeddings'] = df['embeddings'].apply(lambda s: np.array(json.loads(s)))

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26554 entries, 0 to 23905
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Ticker      26554 non-null  object
 1   Name        26554 non-null  object
 2   Type        26554 non-null  object
 3   embeddings  26554 non-null  object
dtypes: object(4)
memory usage: 1.0+ MB


In [96]:
test_df

,Unnamed: 0,Ticker,Name,Type,embeddings
0,0,EURUSD=X,Euro / US Dollar,FOREX,"[0.02847134694457054, 0.008860748261213303, -0..."
1,1,GBPUSD=X,British Pound / US Dollar,FOREX,"[0.027380986139178276, -0.0490446463227272, -0..."
2,2,USDJPY=X,US Dollar / Japanese Yen,FOREX,"[-0.0015552728436887264, 0.013238106854259968,..."
3,3,AUDUSD=X,Australian Dollar / US Dollar,FOREX,"[0.008561188355088234, 0.014089046977460384, -..."
4,4,USDCAD=X,US Dollar / Canadian Dollar,FOREX,"[0.01161902491003275, 0.006832808256149292, -0..."
...,...,...,...,...,...
26549,23900,ZYRX,"Zyrox Mining International, Inc.",STOCK,"[-0.13603752851486206, 0.12183122336864471, -0..."
26550,23902,ZZHGF,"ZhongAn Online P & C Insurance Co., Ltd.",STOCK,"[-0.09154510498046875, 0.12819936871528625, -0..."
26551,23903,ZZHGY,"ZhongAn Online P & C Insurance Co., Ltd.",STOCK,"[-0.09154510498046875, 0.12819936871528625, -0..."
26552,23904,ZZLL,"ZZLL Information Technology, Inc.",STOCK,"[-0.13600338995456696, 0.12652821838855743, -0..."


In [91]:
import ast
df = pd.read_csv('data/tickers_data.csv')
df['embeddings'] = df['embeddings'].apply(lambda x: np.array(ast.literal_eval(x)))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 1)

In [121]:
import yfinance as yf

# Créer un objet Ticker pour Apple
apple = yf.Ticker("AAPL")

# Récupérer les informations fondamentales
info = apple.info

# Afficher quelques informations clés
print("Nom de l'entreprise :", info.get("longName"))
print("Secteur :", info.get("sector"))
print("Industrie :", info.get("industry"))
print("Capitalisation boursière :", info.get("marketCap"))
print("Résumé de l'activité :", info.get("longBusinessSummary"))

Nom de l'entreprise : Apple Inc.
Secteur : Technology
Industrie : Consumer Electronics
Capitalisation boursière : 3688676720640
PER (Price Earnings Ratio) : 38.91442
Rendement du dividende : 0.41
Résumé de l'activité : Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, su